Origin : http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html  
Translator : Hongpu Liu

# Autograd：自动微分
对于PyTorch中所有神经网络来说，最核心的就是``autograd``包。让我们首先简单看一下它的使用，然后训练我们第一个神经网络。

``autograd``包对所有张量操作提供了自动微分功能。它是一种运行时定义框架，意味着你的反向传播在运行时定义，因此每次迭代都可以不同。

**补充：PyTorch是动态图，运行时建图；而静态图在运行前需要编译，运行时图的结构是不能改变的。**

下面通过几个例子来展示其中概念。

## 1. 变量（Variable）
**autograd.Variable**是**autograd**的核心类。它包含张量，并支持集合所有定义在张量之上的操作。一旦但成了计算，就可以调用**.backward()**自动计算梯度。

可以通过**.data**属性来调用Variable中的张量，而关于变量的梯度将会累加到**.grad**属性中。

![](./imgs/AutoGrade1.png)

**Function**是实现自动梯度功能另外一个重要的类。

**Variable**和**Function**互相连接来构建一个无环图，该无环图将所有计算过程进行编码。每个变量有一个**.grad_fn**属性，该属性用来引用创建**Variable**的**Function**（除了用户创建的变量，用户创建变量的**grad_fn**是**None**）。

如果需要计算微分，可以调用**Variable**的**.backward()**。如果**Variable**是标量（只包含一个数据元素），不需要为**.backward()**指定任何参数。反之，需要制定**gradient**参数的值，需要将一个与梯度维度相同的张量传递给它。

In [1]:
from __future__ import print_function
import torch
from torch.autograd import Variable

创建变量：

In [2]:
x = Variable(torch.ones(2,2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



对变量做一个操作：

In [3]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



**y**是由操作创建的结果，因此它有一个**grad_fn**。

In [4]:
print(y.grad_fn)

对**y**做更多的操作：

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



## 2. 梯度（Gradients）
现在来进行反向传播，**out.backward()**等价于**out.backward(torch.Tensor([1.0]))**。

In [6]:
out.backward()

打印梯度**d(out)/dx**：

In [7]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



我们的到一个矩阵所有值为**4.5**。
设**out**变量为$o$，有：

前馈过程：
$$o = \frac{1}{4}\sum_i z_i$$
$$z_i = 3(x_i+2)^2$$
$$z_i\bigr\rvert_{x_i=1} = 27$$

梯度：
$$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$$
$$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$$

还可以利用**autograd**做更疯狂的事情！

In [8]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print(y)

Variable containing:
 1665.5319
  291.7892
  -88.4084
[torch.FloatTensor of size 3]



**补充**  

若调用为*y.backward(gradient=grads)*，表示：

$$\frac {\partial {obj}} {\partial z} \frac {\partial z} {\partial \omega} = grads \times \frac {\partial z} {\partial \omega}$$


In [9]:
# 因为y是一个向量函数，因此在backward是要传递gradient参数，参数中对应的值表示其梯度的比例
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])

# 注意每次backward产生的梯度都会累加到x.grad上，因此反复执行这个代码块，会看到梯度在不断增加
y.backward(gradients)

print(x.grad)

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]



**补充阅读**

**Variable**和**Function**的文档可以参阅：[http://pytorch.org/docs/autograd](http://pytorch.org/docs/autograd)